In [ ]:
!rm -Rf sample_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Data

In [ ]:
!cp '/content/drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_img_txt_words_senti_train.pkl' .

In [ ]:
! pip -q install optuna
! pip -q install tensorflow-addons

     |████████████████████████████████| 302 kB 5.1 MB/s 
     |████████████████████████████████| 80 kB 7.1 MB/s 
     |████████████████████████████████| 208 kB 44.7 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 111 kB 48.7 MB/s 
     |████████████████████████████████| 141 kB 54.0 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
import pickle
import numpy as np

with open('memotion_img_txt_words_senti_train.pkl', 'rb') as f:
    train_values = np.array(list(pickle.load(f).values()), dtype='object')
train_im, train_txt, train_labels = train_values[:, 0], train_values[:, 1], train_values[:, 2]
train_im = np.array(list(train_im))[:,0,:]
train_txt = np.array(list(train_txt))[:,0,:]
train_labels = np.array(list(train_labels))
train_im.shape, train_txt.shape, train_labels.shape

((45297, 4096), (45297, 300), (45297,))

# k-Folds

In [ ]:
from sklearn.model_selection import KFold

X = [(i[0], i[1]) for i in zip(train_im, train_txt)]
y = train_labels

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=21)
kf

KFold(n_splits=5, random_state=21, shuffle=True)

In [ ]:
X_splits = []
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in train_index:
        X_train.append(X[i])
        y_train.append(y[i])
    for i in test_index:
        X_test.append(X[i])
        y_test.append(y[i])
    X_splits.append([np.array(X_train, dtype='object'), np.array(X_test, dtype='object'), np.array(y_train, dtype='object'), np.array(y_test, dtype='object')])
    # print(X_train, X_test)

# Sentiment Analysis

In [ ]:
from time import time
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
import os 
import cv2
import shutil
import random
import optuna  

import tensorflow as tf
import tensorflow.keras as K
import tensorflow_addons as tfa
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as bk

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split


from tensorboard.plugins import projector

# if tf.test.gpu_device_name()!='':
#   physical_devices = tf.config.list_physical_devices('GPU')
#   tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/Wipro/Final/Base_Models/models/stanford_scratch_memotion_words_basic_model.hdf5'

In [ ]:
basic_model = K.models.load_model(MODEL_PATH, compile=False)
# basic_model.summary();

In [ ]:
def custom_model_using_pretrained_stanford(model=basic_model):
  model.trainable = False
  for layer in model.layers:
      layer.trainable =  False
  inp1 = model.layers[0].input
  inp2 = model.layers[1].input

  out1 = model.layers[-2].output
  out2 = model.layers[-1].output

  concat_x = tf.keras.layers.Concatenate()([out1,out2])
  temp_layer = tf.keras.layers.Dense(256,activation='tanh', kernel_regularizer='l2', name='dense_a')(concat_x)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_a')(temp_layer)
  temp_layer = tf.keras.layers.Dense(256,activation='tanh', name='dense_b')(concat_x)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_b')(temp_layer)
  temp_layer = tf.keras.layers.Dense(256,activation='relu',name='dense_c')(concat_x)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_c')(temp_layer)
  temp_layer = tf.keras.layers.Dense(3,activation='softmax',name='dense_d')(temp_layer)

  senti_model = tf.keras.Model(inputs=[inp1,inp2],outputs= temp_layer)

  return senti_model

In [ ]:
CROSS_VALIDATION_SCORES = {}

In [ ]:
current_split = 0
for X_train, X_test, y_train, y_test in X_splits:
    X_img_train = np.array([i for i in X_train[:, 0]])
    X_text_train = np.array([i for i in X_train[:, 1]])
    X_img_test = np.array([i for i in X_test[:, 0]])
    X_text_test = np.array([i for i in X_test[:, 1]])
    y_train = np.array([int(i) for i in y_train])
    y_test = np.array([int(i) for i in y_test])
    
    current_split += 1
    print(f'\n\n----------------------------------------TRAININ SPLIT {current_split}----------------------------------------')
    senti_model = custom_model_using_pretrained_stanford()
    senti_model.compile(
        loss = K.losses.SparseCategoricalCrossentropy(),
        optimizer = K.optimizers.Adam(learning_rate=6e-7),
        metrics = ['acc', 'mse']
    )
    hist = senti_model.fit(
        x=[X_img_train,X_text_train],
        y=y_train,
        batch_size=256,
        epochs=250,
        )
    
    y_pred = senti_model.predict([X_img_test,X_text_test])
    score = f1_score(tf.cast(y_test,tf.int64),tf.math.argmax(y_pred,axis=-1,output_type=tf.int64),average='macro')
    acc = accuracy_score(tf.cast(y_test,tf.int64),tf.math.argmax(y_pred,axis=-1,output_type=tf.int64))
    CROSS_VALIDATION_SCORES[current_split] = (senti_model, score)
    print(f'\nTest Accuracy: {acc}')
    print(f'F1 Score: {score}')
    
    del senti_model
    # break



----------------------------------------TRAININ SPLIT 1----------------------------------------
Epoch 1/250
142/142 [==============================] - 3s 12ms/step - loss: 1.4424 - acc: 0.3416 - mse: 1.8571
Epoch 2/250
142/142 [==============================] - 2s 12ms/step - loss: 1.4363 - acc: 0.3421 - mse: 1.8566
Epoch 3/250
142/142 [==============================] - 2s 11ms/step - loss: 1.4154 - acc: 0.3440 - mse: 1.8548
Epoch 4/250
142/142 [==============================] - 2s 12ms/step - loss: 1.4173 - acc: 0.3403 - mse: 1.8548
Epoch 5/250
142/142 [==============================] - 2s 11ms/step - loss: 1.4167 - acc: 0.3384 - mse: 1.8537
Epoch 6/250
142/142 [==============================] - 2s 11ms/step - loss: 1.4057 - acc: 0.3413 - mse: 1.8530
Epoch 7/250
142/142 [==============================] - 2s 11ms/step - loss: 1.3989 - acc: 0.3440 - mse: 1.8522
Epoch 8/250
142/142 [==============================] - 2s 11ms/step - loss: 1.3973 - acc: 0.3410 - mse: 1.8519
Epoch 9/250
14

In [ ]:
avg_score = sum([i[-1] for i in CROSS_VALIDATION_SCORES.values()]) / current_split

best_split = sorted(CROSS_VALIDATION_SCORES, key=lambda x: CROSS_VALIDATION_SCORES[x][-1], reverse=True)[0]
best_model, best_score = CROSS_VALIDATION_SCORES[best_split]

In [ ]:
avg_score, best_score

(0.373450568356483, 0.3820248834974401)

In [ ]:
best_model.save('/content/drive/MyDrive/Wipro/Final/kFold/Task_A/models/kfold_memotion_words_stanford_memotion_taska_best_model.hdf5')